In [4]:
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
import statistics as st
import seaborn as sns
import datetime 
import pandas as pd
import os
import sqlite3
from sqlite3 import Error
from sqlalchemy import create_engine

In [5]:
# Re-load the datasets after execution state reset
superstore_file_path = "Sample-Superstore.csv"

# Read the datasets
superstore_df = pd.read_csv(superstore_file_path, encoding="ISO-8859-1")


# Ensure numerical columns are correctly interpreted for superstore data
numeric_columns = ["Sales", "Profit", "Discount", "Quantity"]
for col in numeric_columns:
    if col in superstore_df.columns:
        superstore_df[col] = pd.to_numeric(superstore_df[col], errors="coerce")

# 1) Add calculated columns to Sample-Superstore
superstore_df["Estimated Cost"] = superstore_df["Sales"] - (superstore_df["Profit"] + (superstore_df["Sales"] * superstore_df["Discount"]))
superstore_df["Estimated Cost p/unit"] = superstore_df["Estimated Cost"] / superstore_df["Quantity"]
superstore_df["Estimated Sale Price p/unit"] = superstore_df["Sales"] / superstore_df["Quantity"]
superstore_df["Estimated Profit p/unit"] = superstore_df["Estimated Sale Price p/unit"] - superstore_df["Estimated Cost p/unit"]
superstore_df["Estimated Profit Margin p/unit"] = superstore_df["Estimated Profit p/unit"] / superstore_df["Estimated Sale Price p/unit"]

# 2) Add projected profit adjustments and pricing corrections
# Define the minimum projected profit margin requirement
minimum_profit_margin = 0.10  # 10%

# Initialize adjusted discount column
superstore_df["Adjusted Discount"] = superstore_df["Discount"]

# Cases where applying a discount reduction can achieve 10% profit margin
profit_target_sales = superstore_df["Estimated Cost"] / (1 - minimum_profit_margin)
superstore_df["Adjusted Discount"] = 1 - (profit_target_sales / superstore_df["Sales"])

# Ensure the adjusted discount is not negative (meaning no discount applied if it's already profitable)
superstore_df["Adjusted Discount"] = superstore_df["Adjusted Discount"].clip(lower=0)

# Identify cases where discount adjustment alone is not enough (price adjustment needed)
needs_price_adjustment = superstore_df["Estimated Profit Margin p/unit"] < 0.1

# Assign actions based on whether discount or price needs adjusting
superstore_df["Action"] = "Adjust Discount"
superstore_df.loc[needs_price_adjustment, "Action"] = "Adjust Price"

# Adjust Sale Price Per Unit to meet the 10% profit margin requirement
superstore_df.loc[needs_price_adjustment, "Adjusted Sale Price p/unit"] = superstore_df["Estimated Cost p/unit"] / (1 - minimum_profit_margin)

# Ensure adjusted sale price is only changed for "Adjust Price" cases
superstore_df["Adjusted Sale Price p/unit"].fillna(superstore_df["Estimated Sale Price p/unit"], inplace=True)

# Recalculate projected profit per unit for adjusted prices
superstore_df["Projected Profit p/unit"] = superstore_df["Adjusted Sale Price p/unit"] - superstore_df["Estimated Cost p/unit"]

# Calculate the required discount to achieve the minimum projected profit margin
superstore_df["Projected Profit Margin"] = superstore_df["Projected Profit p/unit"] / superstore_df["Adjusted Sale Price p/unit"]

# Recalculate projected total profit for adjusted prices
superstore_df["Projected total profit"] = (superstore_df["Adjusted Sale Price p/unit"] * superstore_df["Quantity"]) - superstore_df["Estimated Cost"]

# Calculate projected sales based on adjusted pricing
superstore_df["Projected Sales"] = superstore_df["Adjusted Sale Price p/unit"] * superstore_df["Quantity"]




/var/folders/kj/_mqw0wd165b_qz356cmtmkzc0000gn/T/ipykernel_6204/3026991110.py:46: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  superstore_df["Adjusted Sale Price p/unit"].fillna(superstore_df["Estimated Sale Price p/unit"], inplace=True)


In [6]:
negative_profit_transactions = superstore_df[superstore_df["Profit"]<0]
negative_profit_transactions

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Estimated Sale Price p/unit,Estimated Profit p/unit,Estimated Profit Margin p/unit,Adjusted Discount,Action,Adjusted Sale Price p/unit,Projected Profit p/unit,Projected Profit Margin,Projected total profit,Projected Sales
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,191.5155,9.575775,0.050000,0.000000,Adjust Price,202.155250,20.215525,0.100000,101.077625,1010.776250
14,15,US-2015-118983,11/22/2015,11/26/2015,Standard Class,HP-14815,Harold Pawlan,Home Office,United States,Fort Worth,...,13.7620,-13.762000,-1.000000,0.000000,Adjust Price,30.582222,3.058222,0.100000,15.291111,152.911111
15,16,US-2015-118983,11/22/2015,11/26/2015,Standard Class,HP-14815,Harold Pawlan,Home Office,United States,Fort Worth,...,0.8480,-0.593600,-0.700000,0.000000,Adjust Price,1.601778,0.160178,0.100000,0.480533,4.805333
23,24,US-2017-156909,7/16/2017,7/18/2017,Second Class,SF-20065,Sandra Flanagan,Consumer,United States,Philadelphia,...,35.6860,10.196000,0.285714,0.206349,Adjust Discount,35.686000,10.196000,0.285714,20.392000,71.372000
27,28,US-2015-150630,9/17/2015,9/21/2015,Standard Class,TB-21520,Tracy Blumstein,Consumer,United States,Philadelphia,...,440.4900,-17.619600,-0.040000,0.000000,Adjust Price,509.010667,50.901067,0.100000,356.307467,3563.074667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9920,9921,CA-2016-149272,3/15/2016,3/19/2016,Standard Class,MY-18295,Muhammed Yedwab,Corporate,United States,Bryan,...,3.1980,-2.558400,-0.800000,0.000000,Adjust Price,6.396000,0.639600,0.100000,4.477200,44.772000
9921,9922,CA-2014-111360,11/24/2014,11/30/2014,Standard Class,AT-10435,Alyssa Tate,Home Office,United States,Akron,...,1.9140,-0.191400,-0.100000,0.000000,Adjust Price,2.339333,0.233933,0.100000,0.701800,7.018000
9931,9932,CA-2015-104948,11/13/2015,11/17/2015,Standard Class,KH-16510,Keith Herrera,Consumer,United States,San Bernardino,...,170.8330,15.575950,0.091176,0.000000,Adjust Price,172.507833,17.250783,0.100000,69.003133,690.031333
9937,9938,CA-2016-164889,6/3/2016,6/6/2016,Second Class,CP-12340,Christine Phan,Corporate,United States,Los Angeles,...,35.5440,6.220200,0.175000,0.083333,Adjust Discount,35.544000,6.220200,0.175000,12.440400,71.088000


In [7]:
# Recalculate the "Action" column to correctly account for cases where reducing the discount achieves a 10% total profit margin

# Define the minimum projected profit margin requirement
minimum_profit_margin = 0.10  # 10%

# Recalculate profit target sales to achieve a 10% **total** profit margin
profit_target_sales_total = superstore_df["Estimated Cost"] / (1 - minimum_profit_margin)

# Determine cases where reducing discount would be enough to meet the required total profit margin
adjust_discount_needed = (1 - (profit_target_sales_total / superstore_df["Sales"])) < superstore_df["Discount"]

# Identify cases where price adjustment is still needed
needs_price_adjustment = ~adjust_discount_needed & (superstore_df["Estimated Profit Margin p/unit"] < 0.1)

# Assign "Adjust Discount" where reducing the discount is sufficient
superstore_df["Action"] = "No Action Needed"
superstore_df.loc[adjust_discount_needed, "Action"] = "Adjust Discount"
superstore_df.loc[needs_price_adjustment, "Action"] = "Adjust Price"

# Adjust the discount for the cases where discount reduction is needed
superstore_df.loc[adjust_discount_needed, "Adjusted Discount"] = 1 - (profit_target_sales_total / superstore_df["Sales"])

# Ensure the adjusted discount is not negative
superstore_df["Adjusted Discount"] = superstore_df["Adjusted Discount"].clip(lower=0)

# Adjust Sale Price Per Unit only where necessary
superstore_df.loc[needs_price_adjustment, "Adjusted Sale Price p/unit"] = superstore_df["Estimated Cost p/unit"] / (1 - minimum_profit_margin)

# Ensure adjusted sale price is only changed for "Adjust Price" cases
superstore_df["Adjusted Sale Price p/unit"].fillna(superstore_df["Estimated Sale Price p/unit"], inplace=True)

# Recalculate projected profit per unit for adjusted prices
superstore_df["Projected Profit p/unit"] = superstore_df["Adjusted Sale Price p/unit"] - superstore_df["Estimated Cost p/unit"]

# Calculate the required discount to achieve the minimum projected profit margin
superstore_df["Projected Profit Margin"] = superstore_df["Projected Profit p/unit"] / superstore_df["Adjusted Sale Price p/unit"]

# Recalculate projected total profit for adjusted prices
superstore_df["Projected total profit"] = (superstore_df["Adjusted Sale Price p/unit"] * superstore_df["Quantity"]) - superstore_df["Estimated Cost"]

# Calculate projected sales based on adjusted pricing
superstore_df["Projected Sales"] = superstore_df["Adjusted Sale Price p/unit"] * superstore_df["Quantity"]



/var/folders/kj/_mqw0wd165b_qz356cmtmkzc0000gn/T/ipykernel_6204/2106287047.py:30: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  superstore_df["Adjusted Sale Price p/unit"].fillna(superstore_df["Estimated Sale Price p/unit"], inplace=True)


In [8]:
superstore_df["Transaction Profit Margin"] = superstore_df ["Profit"]/ superstore_df["Sales"]

In [9]:
superstore_df.columns

Index(['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State',
       'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category',
       'Product Name', 'Sales', 'Quantity', 'Discount', 'Profit',
       'Estimated Cost', 'Estimated Cost p/unit',
       'Estimated Sale Price p/unit', 'Estimated Profit p/unit',
       'Estimated Profit Margin p/unit', 'Adjusted Discount', 'Action',
       'Adjusted Sale Price p/unit', 'Projected Profit p/unit',
       'Projected Profit Margin', 'Projected total profit', 'Projected Sales',
       'Transaction Profit Margin'],
      dtype='object')

In [10]:
superstore_df

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Estimated Profit p/unit,Estimated Profit Margin p/unit,Adjusted Discount,Action,Adjusted Sale Price p/unit,Projected Profit p/unit,Projected Profit Margin,Projected total profit,Projected Sales,Transaction Profit Margin
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,20.956800,0.1600,0.066667,No Action Needed,130.98000,20.956800,0.1600,41.913600,261.96000,0.1600
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,73.194000,0.3000,0.222222,No Action Needed,243.98000,73.194000,0.3000,219.582000,731.94000,0.3000
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,3.435700,0.4700,0.411111,No Action Needed,7.31000,3.435700,0.4700,6.871400,14.62000,0.4700
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,9.575775,0.0500,0.000000,Adjust Discount,202.15525,20.215525,0.1000,101.077625,1010.77625,-0.4000
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,3.495000,0.3125,0.236111,No Action Needed,11.18400,3.495000,0.3125,6.990000,22.36800,0.1125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,9990,CA-2014-110422,1/21/2014,1/23/2014,Second Class,TB-21400,Tom Boeckenhauer,Consumer,United States,Miami,...,3.050800,0.3625,0.291667,No Action Needed,8.41600,3.050800,0.3625,9.152400,25.24800,0.1625
9990,9991,CA-2017-121258,2/26/2017,3/3/2017,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,7.816600,0.1700,0.077778,No Action Needed,45.98000,7.816600,0.1700,15.633200,91.96000,0.1700
9991,9992,CA-2017-121258,2/26/2017,3/3/2017,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,35.554200,0.2750,0.194444,Adjust Discount,129.28800,35.554200,0.2750,71.108400,258.57600,0.0750
9992,9993,CA-2017-121258,2/26/2017,3/3/2017,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,3.330000,0.4500,0.388889,No Action Needed,7.40000,3.330000,0.4500,13.320000,29.60000,0.4500


In [11]:
superstore_df = superstore_df[['Order ID', 'Order Date', 'Ship Date', 'Ship Mode',
       'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State',
       'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category',
       'Product Name', 'Sales', 'Quantity', 'Discount', 'Profit','Transaction Profit Margin',
       'Estimated Cost', 'Estimated Cost p/unit',
       'Estimated Sale Price p/unit', 'Estimated Profit p/unit',
       'Estimated Profit Margin p/unit', 'Adjusted Discount', 'Action',
       'Adjusted Sale Price p/unit', 'Projected Profit p/unit',
       'Projected Profit Margin', 'Projected total profit', 'Projected Sales',
       ]]

In [12]:
superstore_df["Transaction Profit Margin"]

0       0.1600
1       0.3000
2       0.4700
3      -0.4000
4       0.1125
         ...  
9989    0.1625
9990    0.1700
9991    0.0750
9992    0.4500
9993    0.3000
Name: Transaction Profit Margin, Length: 9994, dtype: float64

In [13]:
superstore_df.to_csv("superstore_update.csv")


In [14]:
#TODO - check discount of same items in different states
# do another kpi for sale price

In [16]:


# # Ensure numerical columns are correctly interpreted
# numeric_columns = ["Sales", "Quantity"]
# for col in numeric_columns:
#     if col in superstore_df.columns:
#         superstore_df[col] = pd.to_numeric(superstore_df[col], errors="coerce")

# Convert "Order Date" to datetime format if available
if "Order Date" in superstore_df.columns:
    superstore_df["Order Date"] = pd.to_datetime(superstore_df["Order Date"], errors="coerce")
    superstore_df["Year-Month"] = superstore_df["Order Date"].dt.to_period("M")

# Group data by Year-Month and calculate Price per Unit
price_per_unit_kpi = superstore_df.groupby("Year-Month").agg(
    Total_Sales=("Sales", "sum"),
    Total_Quantity=("Quantity", "sum")
).reset_index()

# Calculate Price per Unit
price_per_unit_kpi["Price per Unit"] = price_per_unit_kpi["Total_Sales"] / price_per_unit_kpi["Total_Quantity"]




/var/folders/kj/_mqw0wd165b_qz356cmtmkzc0000gn/T/ipykernel_6204/1140624521.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  superstore_df["Order Date"] = pd.to_datetime(superstore_df["Order Date"], errors="coerce")


In [18]:
price_per_unit_kpi.head()

,Year-Month,Total_Sales,Total_Quantity,Price per Unit
0,2014-01,14236.895,284,50.129912
1,2014-02,4519.892,159,28.426994
2,2014-03,55691.009,585,95.198306
3,2014-04,28295.345,536,52.789823
4,2014-05,23648.287,466,50.747397


# checking profit margin by subcategory

In [ ]:
# 1) Profitability Breakdown Within Furniture
furniture_df = superstore_df[superstore_df["Category"] == "Furniture"]

# Aggregate profit margin by sub-category
furniture_subcategory_profit = furniture_df.groupby("Sub-Category").agg(
    {"Sales": "sum", "Profit": "sum", "Discount": "mean"}
).reset_index()

# Calculate profit margin percentage
furniture_subcategory_profit["Profit Margin (%)"] = (furniture_subcategory_profit["Profit"] / furniture_subcategory_profit["Sales"]) * 100


furniture_subcategory_profit

In [20]:
technology_df = superstore_df[superstore_df["Category"] == "Technology"]
technology_discount_correlation = technology_df[["Transaction Profit Margin", "Discount"]].corr().iloc[0, 1]


In [23]:
# Aggregate profit margin by sub-category
technology_subcategory_profit = technology_df.groupby("Sub-Category").agg(
    {"Sales": "sum", "Profit": "sum", "Discount": "mean"}
).reset_index()

# Calculate profit margin percentage
technology_subcategory_profit["Profit Margin (%)"] = (technology_subcategory_profit["Profit"] / technology_subcategory_profit["Sales"]) * 100

In [24]:
technology_subcategory_profit

,Sub-Category,Sales,Profit,Discount,Profit Margin (%)
0,Accessories,167380.318,41936.6357,0.078452,25.054700
1,Copiers,149528.030,55617.8249,0.161765,37.195585
2,Machines,189238.631,3384.7569,0.306087,1.788618
3,Phones,330007.054,44515.7306,0.154556,13.489327


In [21]:
technology_discount_correlation

np.float64(-0.7551586032184515)

In [22]:
office_supplies_df = superstore_df[superstore_df["Category"] == "Office Supplies"]
office_supplies_discount_correlation = office_supplies_df[["Transaction Profit Margin", "Discount"]].corr().iloc[0, 1]
office_supplies_discount_correlation

np.float64(-0.8772941598235355)

In [25]:
# Aggregate profit margin by sub-category
office_supplies_subcategory_profit = office_supplies_df.groupby("Sub-Category").agg(
    {"Sales": "sum", "Profit": "sum", "Discount": "mean"}
).reset_index()

# Calculate profit margin percentage
office_supplies_subcategory_profit["Profit Margin (%)"] = (office_supplies_subcategory_profit["Profit"] / office_supplies_subcategory_profit["Sales"]) * 100

In [26]:
office_supplies_subcategory_profit

,Sub-Category,Sales,Profit,Discount,Profit Margin (%)
0,Appliances,107532.161,18138.0054,0.166524,16.867517
1,Art,27118.792,6527.7870,0.074874,24.071083
2,Binders,203412.733,30221.7633,0.372292,14.857361
3,Envelopes,16476.402,6964.1767,0.080315,42.267582
4,Fasteners,3024.280,949.5182,0.082028,31.396504
5,Labels,12486.312,5546.2540,0.068681,44.418672
6,Paper,78479.206,34053.5693,0.074891,43.391837
7,Storage,223843.608,21278.8264,0.074704,9.506113
8,Supplies,46673.538,-1189.0995,0.076842,-2.547695


In [27]:
# profit by state
profit_by_state = superstore_df.groupby("State")["Profit"].sum()

In [28]:
profit_by_state

State
Alabama                  5786.8253
Arizona                 -3427.9246
Arkansas                 4008.6871
California              76381.3871
Colorado                -6527.8579
Connecticut              3511.4918
Delaware                 9977.3748
District of Columbia     1059.5893
Florida                 -3399.3017
Georgia                 16250.0433
Idaho                     826.7231
Illinois               -12607.8870
Indiana                 18382.9363
Iowa                     1183.8119
Kansas                    836.4435
Kentucky                11199.6966
Louisiana                2196.1023
Maine                     454.4862
Maryland                 7031.1788
Massachusetts            6785.5016
Michigan                24463.1876
Minnesota               10823.1874
Mississippi              3172.9762
Missouri                 6436.2105
Montana                  1833.3285
Nebraska                 2037.0942
Nevada                   3316.7659
New Hampshire            1706.5028
New Jersey    

In [31]:
# discount by category
# Aggregate profit margin by sub-category
category_profit = superstore_df.groupby("Category").agg(
    {"Sales": "sum", "Profit": "sum", "Discount": "mean","Estimated Cost p/unit": "mean" }
).reset_index()

# Calculate profit margin percentage
category_profit["Profit Margin (%)"] = (category_profit["Profit"] / category_profit["Sales"]) * 100

In [32]:
category_profit

,Category,Sales,Profit,Discount,Estimated Cost p/unit,Profit Margin (%)
0,Furniture,741999.7953,18451.2728,0.173923,73.433070,2.486695
1,Office Supplies,719047.0320,122490.8008,0.157285,23.005348,17.035158
2,Technology,836154.0330,145454.9481,0.132323,81.793887,17.395712


Discount has a big impact on profit margins, but is not the only factor. High costs p/unit in conjunction with exagerated discounts seems to provide a better explanation. 

In [34]:
# COGS by subcatgory

# Ensure numerical columns are correctly interpreted
numeric_columns = ["Sales", "Estimated Cost"]
for col in numeric_columns:
    if col in superstore_df.columns:
        superstore_df[col] = pd.to_numeric(superstore_df[col], errors="coerce")

# 1️⃣ **Calculate COGS as a Percentage of Sales**
superstore_df["COGS %"] = (superstore_df["Estimated Cost"] / superstore_df["Sales"]) * 100

# 2️⃣ **Aggregate COGS % by Category & Sub-Category**
cogs_percentage_df = superstore_df.groupby(["Category", "Sub-Category"]).agg(
    Total_Sales=("Sales", "sum"),
    Total_Estimated_Cost=("Estimated Cost", "sum")
).reset_index()

# Calculate COGS % for each Category & Sub-Category
cogs_percentage_df["COGS %"] = (cogs_percentage_df["Total_Estimated_Cost"] / cogs_percentage_df["Total_Sales"]) * 100

cogs_percentage_df

,Category,Sub-Category,Total_Sales,Total_Estimated_Cost,COGS %
0,Furniture,Bookcases,114879.9963,97551.961259,84.916404
1,Furniture,Chairs,328449.1030,252044.136600,76.737654
2,Furniture,Furnishings,91705.1640,69937.067600,76.262955
3,Furniture,Tables,206965.5320,180498.816000,87.212018
4,Office Supplies,Appliances,107532.1610,81903.855900,76.166846
5,Office Supplies,Art,27118.7920,18770.174600,69.214641
6,Office Supplies,Binders,203412.7330,129445.890400,63.637064
7,Office Supplies,Envelopes,16476.4020,8338.234900,50.607134
8,Office Supplies,Fasteners,3024.2800,1834.473800,60.658200
9,Office Supplies,Labels,12486.3120,6290.707600,50.380830


In [38]:
# 2️⃣ **Aggregate COGS % by Category & Sub-Category**
cogs_percentage_df_category = superstore_df.groupby(["Category"]).agg(
    Total_Sales=("Sales", "sum"),
    Total_Estimated_Cost=("Estimated Cost", "sum")
).reset_index()

cogs_percentage_df_category["COGS %"] = (cogs_percentage_df_category["Total_Estimated_Cost"] / cogs_percentage_df_category["Total_Sales"]) * 100

cogs_percentage_df_category

,Category,Total_Sales,Total_Estimated_Cost,COGS %
0,Furniture,741999.7953,600031.981459,80.866866
1,Office Supplies,719047.0320,520151.290600,72.338980
2,Technology,836154.0330,568038.432200,67.934664
